In [ ]:
import warnings
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', 200)
warnings.filterwarnings('ignore', category=UserWarning)

# Original Excel source file from the data provider stored on a GCOOS WAF folder
sourcefile = 'https://gcoos4.geos.tamu.edu/WAF/MBON/JuvenileSportFish/2016JuvenileSportfishNOAA.xlsx'

# -- Read the "Sportfish Data" sheet from the Excel file
# --
df_spt = pd.read_excel(sourcefile, sheet_name='Sportfish Data')

# Read the "Collection Stn Data" sheet from the Excel file
# --
df_cln = pd.read_excel(sourcefile, sheet_name='Collection Stn Data')

# -- Read the "Species Code" sheet from the Excel file
# --
df_spc = pd.read_excel(sourcefile, sheet_name='Species Code')

# -- Read the "All Data" sheet from the Excel file
# --
df_all = pd.read_excel(sourcefile, sheet_name='All Data')
# Replace special chars in column names
df_all.columns = [col.replace('%', 'percent') for col in df_all.columns]

# Drop columns that are not needed
df_all.drop(columns=['Cyn (lengths 30-200 mm)','Cyn (lengths 30-200 mm) pres/abs only','Lug (lengths 30-260 mm)  pres/abs only'], inplace=True)
# Rename some columns
df_all = df_all.rename(columns={'Cyn ALL Lengths': 'cyn', 'Lug (lengths 30-260 mm)': 'lug'})
# List of count columns to rename and melt:
count_cols = [ 'Ari','Arp','Bab','Chf','cyn','Epi','Has','Hie','Hpa','Hyp','Lam','Lar','lug','Lum','Lun','Lus','Mym','Occ','Paa','Pab','Pal','Poc','Scb','Sco','Sev','Spa']
# Make all lower case (to match with species code list later)
for acol in count_cols:
    df_all = df_all.rename(columns={acol: acol.lower()})
counts_lower = [item.lower() for item in count_cols]

# -- Melt the dataframe to long form
# --
# Melt species count columns to just SpeciesCode and SpeciesCount)
df_long = pd.melt(df_all,
                  id_vars=[col for col in df_all.columns if col not in counts_lower], # other columns to keep
                  value_vars=counts_lower,        # species columns
                  var_name='SpeciesCode', 
                  value_name='SpeciesCount')

# -- Fix Time
# -- 
# Fill missing times with a default
df_long['Time'] = df_long['Time'].fillna(1200)
# Convert to integer (from float)
df_long['Time'] = df_long['Time'].astype('int')

# -- Add a Datetime field (date and time)
# --
df_long['timestr'] = df_long['Time'].astype(str)
df_long['minute'] = df_long['timestr'].str[-2:]
# Clean dirty data: minute has values like "78", should probably be "18"
def replace_second_last_char_if_target(s, new_char, target_char):
    s = str(s)  # Ensure it's a string
    if len(s) < 2 or s[-2] != target_char:
        return s
    return s[:-2] + new_char + s[-1]
# DIRTY DATA: replace second last character in the minute column, '7' with '1'
df_long['minute'] = df_long['minute'].apply(lambda x: replace_second_last_char_if_target(x, '1', '7'))
df_long['hour'] = df_long['timestr'].str[:-2]
df_long['hour'].replace(to_replace="", value="00", inplace=True)
#df_long['Date'] = pd.to_datetime(df_long['Date'])
df_long['Datetime'] = pd.to_datetime(df_long['Date'].dt.strftime('%Y-%m-%d') + ' ' + df_long['hour'] + ':' + df_long['Day'].astype(str) + ' ' + df_long['hour'] + ':' + df_long['minute'])
# Format datetime string ()
df_long['Datetime'] = df_long['Datetime'].dt.strftime('%Y-%m-%dT%H:%M-05')
# Drop the extra fields
df_long.drop(['timestr', 'minute', 'hour', 'Date'], axis=1, inplace=True)

# -- Merge Sportfish data
# -- 
merged_df = pd.merge(df_long, df_spt, left_on=['Station','Year','Month','SpeciesCode'], right_on=['Station','Year','Month','Species Code'], how='left').copy()

# Drop unnecessary columns
###drop_these = ['Zone_y','Species Code','Common Name', 'Code']
###merged_df.drop(columns=drop_these, inplace=True)


# Re-rename Zone column that was renamed by merge
merged_df = merged_df.rename(columns={'Zone_x': 'Zone'})

# -- Merge species code data 
# --
merged_df2 = pd.merge(merged_df, df_spc, left_on=['SpeciesCode'], right_on=['Code'], how='left').copy()

# Drop unnecessary columns
drop_these = ['Zone_y','Species Code','Common Name', 'Code']
merged_df2.drop(columns=drop_these, inplace=True)


# -- New dataframe: Group by Keyfield, add up total SpeciesCount 
# --
grouped = merged_df2.groupby('Keyfield').agg({
    'Keyfield': 'first',
     'Station': 'first',
     'Location': 'first',
     'Latitude': 'first',
     'Longitude': 'first',
     'Year': 'first',
     'Month': 'first',
     'Day': 'first',
     'Time': 'first',
     'Zone': 'first',
     'Area Towed': 'first',
     'Salinity': 'first',
     'Temperature': 'first',
     'Grass  percent Cover rep 1': 'first',
     'Grass  percent Cover rep 2': 'first',
     'Grass  percent Cover rep 3': 'first',
     'Grass  percent Cover rep 4': 'first',
     'Grass  percent Cover rep 5': 'first',
     'Grass  percent Cover rep 6': 'first',
     'Grass  percent Cover rep 7': 'first',
     'Grass  percent Cover rep 8': 'first',
     'Grass  percent Cover rep 9': 'first',
     'T  percent Cover rep 1': 'first',
     'T  percent Cover rep 2': 'first',
     'T  percent Cover rep 3': 'first',
     'T  percent Cover rep 4': 'first',
     'T  percent Cover rep 5': 'first',
     'T  percent Cover rep 6': 'first',
     'T  percent Cover rep 7': 'first',
     'T  percent Cover rep 8': 'first',
     'T  percent Cover rep 9': 'first',
     'T Canopy Height rep 1': 'first',
     'T Canopy Height rep 2': 'first',
     'T Canopy Height rep 3': 'first',
     'T Canopy Height rep 4': 'first',
     'T Canopy Height rep 5': 'first',
     'T Canopy Height rep 6': 'first',
     'T Canopy Height rep 7': 'first',
     'T Canopy Height rep 8': 'first',
     'T Canopy Height rep 9': 'first',
     'S  percent Cover rep 1': 'first',
     'S  percent Cover rep 2': 'first',
     'S  percent Cover rep 3': 'first',
     'S  percent Cover rep 4': 'first',
     'S  percent Cover rep 5': 'first',
     'S  percent Cover rep 6': 'first',
     'S  percent Cover rep 7': 'first',
     'S  percent Cover rep 8': 'first',
     'S  percent Cover rep 9': 'first',
     'S Canopy Height rep 1': 'first',
     'S Canopy Height rep 2': 'first',
     'S Canopy Height rep 3': 'first',
     'S Canopy Height rep 4': 'first',
     'S Canopy Height rep 5': 'first',
     'S Canopy Height rep 6': 'first',
     'S Canopy Height rep 7': 'first',
     'S Canopy Height rep 8': 'first',
     'S Canopy Height rep 9': 'first',
     'H  percent Cover rep 1': 'first',
     'H  percent Cover rep 2': 'first',
     'H  percent Cover rep 3': 'first',
     'H  percent Cover rep 4': 'first',
     'H  percent Cover rep 5': 'first',
     'H  percent Cover rep 6': 'first',
     'H  percent Cover rep 7': 'first',
     'H  percent Cover rep 8': 'first',
     'H  percent Cover rep 9': 'first',
     'H Canopy Height Rep 1': 'first',
     'H Canopy Height Rep 2': 'first',
     'H Canopy Height Rep 3': 'first',
     'H Canopy Height Rep 4': 'first',
     'H Canopy Height Rep 5': 'first',
     'H Canopy Height Rep 6': 'first',
     'H Canopy Height Rep 7': 'first',
     'H Canopy Height Rep 8': 'first',
     'H Canopy Height Rep 9': 'first',
     'SAV percent Cover rep 1': 'first',
     'SAV percent Cover rep 2': 'first',
     'SAV percent Cover rep 3': 'first',
     'SAV percent Cover rep 4': 'first',
     'SAV percent Cover rep 5': 'first',
     'SAV percent Cover rep 6': 'first',
     'SAV percent Cover rep 7': 'first',
     'SAV percent Cover rep 8': 'first',
     'SAV percent Cover rep 9': 'first',
     'Total percent Cover rep 1': 'first',
     'Total percent Cover rep 2': 'first',
     'Total percent Cover rep 3': 'first',
     'Total percent Cover rep 4': 'first',
     'Total percent Cover rep 5': 'first',
     'Total percent Cover rep 6': 'first',
     'Total percent Cover rep 7': 'first',
     'Total percent Cover rep 8': 'first',
     'Total percent Cover rep 9': 'first',
     'Algae percent Cover rep 1': 'first',
     'Algae percent Cover rep 2': 'first',
     'Algae percent Cover rep 3': 'first',
     'Algae percent Cover rep 4': 'first',
     'Algae percent Cover rep 5': 'first',
     'Algae percent Cover rep 6': 'first',
     'Algae percent Cover rep 7': 'first',
     'Algae percent Cover rep 8': 'first',
     'Algae percent Cover rep 9': 'first',
     'Cyn Density': 'first',
     'Average Grass percent Cover': 'first',
     'Average T percent Cover': 'first',
     'Average S percent Cover': 'first',
     'Average H percent Cover': 'first',
     'Average T Canopy Height': 'first',
     'Average S Canopy Height': 'first',
     'Average H Canopy Height': 'first',
     'SpeciesCode': 'first',
     'SpeciesCount': 'sum',
     'Datetime': 'first',
     'Length (mm)': 'sum',
     'Species': 'first',
     'Common name': 'first'
})

# Save Lines where SpeciesCount is zero: no need to have duplicates of these in the data
zerocounts_df = grouped[grouped['SpeciesCount'] == 0].copy()

# -- 1) Delete from the merged dataframe all lines with Keyfield value found in zerocounts_df 
# -- 2) Delete from the merged dataframe all lines with SpeciesCount == 0
# -- 3) Concatenate back in one line per Keyfield that had zero total SpeciesCount (to preserve )
# -- (No added value as all counts are 0)
# -- 
values_to_drop = zerocounts_df['Keyfield'].unique()
merged_df2.drop(merged_df2[merged_df2['Keyfield'].isin(values_to_drop)].index, inplace=True)
# Delete from the merged dataframe other lines with zero counts (some species in the sample are present)
merged_df2.drop(merged_df2[merged_df2['SpeciesCount'] == 0].index, inplace=True)
# Concatenate with the zero count dataframe (to keep the benthic coverage data, but just one line per Keyfield / sample)
concat_df =  pd.concat([merged_df2, zerocounts_df], ignore_index=True)

# Sort by date/sample
df_sorted = concat_df.sort_values(by='Keyfield').reset_index(drop=True)

In [ ]:
# Rename some columns
df_sorted = df_sorted.rename(columns={'Common name': 'CommonName', 'Length (mm)': 'IndividualLength'})
# print sorted cols, Collection Stn columns -> join 'by' which columns???
df_sorted.columns.to_list()

In [ ]:
df_cln.columns.to_list()

In [ ]:
df_sorted[df_sorted['Keyfield'] == 201508022]

In [ ]:
# ???( merge Collection Stn data????)
# !!! OBS !!!
# - Sportfish Data that is included in Collection Stn already in df_sorted, plus in som cases at least has better coverage in Sportfish sheet!!
# -> Do not duplcate these data (... by reading the same data from Collection Stn data...)
#    -> either REMOVE these data from Collection Stn data before merging to df_sorted..., or
#    -> publish the Collection Stn data separately!???

# Also, for collection Stn data:
# 1) clean the length data: DROP rows where Length NaN
# 2) Make sure length etc column types float!
# 3) IF DATA TO BE ADDED TO SPORTFISH DATA : Rename columns  by adding _Collection to separate from the sportfish length filelds!???

# RECOMMENDATION : HAVE SEPARATE datasets
# 1) (All data + Sportfish lengths) AND (All data + Collection Stn lengths)
# 2) OR:  ( All data (counts) ) plus (Sportfish Data (lengths)) plus (Collection Stn Data (lengths))
#     2.1) Have Keyfield column in all to have a common ID (need to build it for sportfish and collection stn ...)


df_cln

In [ ]:
# --
# Make dictionary for variable descriptions
# --

# Read the sheet into a DataFrame
df_dscrn = pd.read_excel(sourcefile, sheet_name='Header Key')

# Convert the two columns to a dictionary
# Assumption: first column is key, second column is value
descr_dict = dict(zip(df_dscrn.iloc[:,0], df_dscrn.iloc[:,1]))

# Remove keys that no longer exist in the data frame
keys_to_remove = ['Ari', 'Arp', 'Bab', 'Chf', 'Cyn ALL Lengths', 'Cyn (lengths 30-200 mm)', 'Cyn (lengths 30-200 mm) pres/abs only', 'Epi', 'Has', 'Hie', 'Hpa', 'Hyp', 'Lam', 'Lar', 'Lug (lengths 30-260 mm)', 'Lug (lengths 30-260 mm)  pres/abs only', 'Lum', 'Lun', 'Lus', 'Mym', 'Occ', 'Paa', 'Pab', 'Pal', 'Poc', 'Scb', 'Sco', 'Sev', 'Spa']
for key in keys_to_remove:
    descr_dict.pop(key, None)  # Safe: does nothing if key missing
descr_dict_new = {k.replace('%', 'percent'): v.replace('%', 'percent') for k, v in descr_dict.items()}

print(descr_dict_new)

# Add description to the dict for the new variables: SpeciesCode and count

## The descriptions can be used for writing out description field to the ERDDAP datasets XMl snippet
# !!! JATKA !!!

In [ ]:


# Load the two CSVs
df_all = pd.read_csv("AllData.csv")  # Exported from the original Excel sheet
df_stn = pd.read_csv("CollectionData.csv")  # The one you just uploaded

# Merge based on key fields
merged_df = pd.merge(df_all, df_stn, on=["Station", "Date", "Year", "Month", "Day"], how="inner")

# Save the merged result
merged_df.to_csv("Merged_Sportfish_Data.csv", index=False)
